<a href="https://colab.research.google.com/github/niyati-rao/Capstone_Project_LWE/blob/main/Sage_LPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code below requires SageMath, which cannot be simply installed as a package via e.g, pip, to run. Please locally install Sage and copy and run this code.

In [ ]:
from sage.all import GF, PolynomialRing, vector, matrix, SymmetricFunctions
import numpy as np
import math

In [ ]:
def oracle(s, n, m):
    a_vectors = [vector(field, [np.random.randint(0, 2) for _ in range(n)]) for _ in range(m)]
    b_values = []
    incorrect_indices = {np.random.randint(0, m) for _ in range(threshold)}

    for i, a in enumerate(a_vectors):
        dot_product = sum(a[j] * s[j] for j in range(n))
        error = 1 if i in incorrect_indices else 0
        b_values.append((dot_product + error) % 2)

In [ ]:
def key_constraint(polynomial, z, problem_instance):
    for i, (a_i, b_i) in enumerate(problem_instance):
        rewritten_term = sum(a_i[j] * z[j] for j in range(len(z))) + b_i
        polynomial = polynomial.subs(**{f"x{i}": rewritten_term})
    return polynomial

In [ ]:
def solve_groebner(equations, n):
    pr = PolynomialRing(field, tuple(f"x{i}" for i in range(n)), order="lex")
    z = pr.gens()
    f = [pr(e) for e in equations]
    groebner_basis = pr.ideal(f).groebner_basis()
    solution = [int(p.constant_coefficient()) for p in groebner_basis if p.nvariables() == 1 and p.degree() == 1]
    return solution

In [ ]:
n = 15
m = 3
threshold = m // 3

field = GF(2)
pr = PolynomialRing(field, tuple(f"x{i}" for i in range(n)), order="lex")
x = pr.gens()
s = SymmetricFunctions(QQ)
P = 0
for i in range ((m//3)+1, m+1):
    sym = s([1 for i in range(i)]).expand(m, x[Integer(0):m])
    P += pr(sym.map_coefficients(lambda c: field(c)))

s = vector(field, [np.random.randint(0, 2) for _ in range(n)])
print("Secret key:", s)
num_samples = 10* int(sum([math.comb(n, i) for i in range(m)]) * math.pow(2,2*m))
z = [pr.gen(i) for i in range(n)]

equations = []
for _ in range(num_samples):
    instance = oracle(s, n, m)
    eqn = key_constraint(P, z, instance)
    equations.append(eqn)

solution = solve_groebner(equations, n)
print("Recovered key:", solution)